In [ ]:
setwd("/work/postresearch/Shared/Projects/Farbod")

In [4]:
#packrat::init("/work/postresearch/Shared/Projects/Farbod/CaseMix/")
#packrat::snapshot()

Initializing packrat project in directory:
- "/work/postresearch/Shared/Projects/Farbod/CaseMix"

Initialization complete!



In [5]:
packrat::status()


The following packages have been updated in your library, but have not been recorded in packrat:
              library   packrat
    zeallot     0.1.0        NA

Use packrat::snapshot() to record these packages in packrat.


In [7]:
options(repr.matrix.max.rows = 100, repr.matrix.max.cols = 300)
options(repr.plot.width = 20, repr.plot.height = 15)
options(width = 300)

numcores <- 128

library(tidyverse)
library(data.table)
library(fst)
library(comorbidity)
library(reshape)
library(dtplyr)
library(haven)
library(vroom)
library(dplyr)
library(stringr)
library(fastmatch)
`%!in%` <- Negate(`%in%`)

setDTthreads(numcores)

── Attaching packages ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.8     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.1
✔ readr   2.1.2     ✔ forcats 0.5.2
── Conflicts ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose



Attaching package: ‘reshape’


The following object is masked from ‘package:data.table’:

    melt


The following object is masked from ‘package:dplyr’:

   

# Prepare data

## Clean MBSF data

In [26]:
mbsf_2013=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2013.csv" , num_threads = numcores) %>% as.data.table
mbsf_2014=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2014.csv" , num_threads = numcores) %>% as.data.table
mbsf_2015=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2015.csv" , num_threads = numcores) %>% as.data.table
mbsf_2016=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2016.csv" , num_threads = numcores) %>% as.data.table
mbsf_2017=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2017.csv" , num_threads = numcores) %>% as.data.table
mbsf_2018=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2018.csv" , num_threads = numcores) %>% as.data.table
mbsf_2019=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2019.csv" , num_threads = numcores) %>% as.data.table
mbsf_2020=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2020.csv" , num_threads = numcores) %>% as.data.table


New names:
• `1` -> `1...5`
• `1` -> `1...7`
• `0` -> `0...8`
• `0` -> `0...9`
• `0` -> `0...11`
• `0` -> `0...12`
• `3` -> `3...13`
• `3` -> `3...14`
• `3` -> `3...15`
• `3` -> `3...16`
• `3` -> `3...17`
• `3` -> `3...18`
• `3` -> `3...19`
• `3` -> `3...20`
• `3` -> `3...21`
• `3` -> `3...22`
• `3` -> `3...23`
• `3` -> `3...24`
• `C` -> `C...25`
• `C` -> `C...26`
• `C` -> `C...27`
• `C` -> `C...28`
• `C` -> `C...29`
• `C` -> `C...30`
• `C` -> `C...31`
• `C` -> `C...32`
• `C` -> `C...33`
• `C` -> `C...34`
• `C` -> `C...35`
• `C` -> `C...36`
• `12` -> `12...37`
• `12` -> `12...38`
• `12` -> `12...39`
• `0` -> `0...40`
• `` -> `...41`
• `` -> `...42`
Rows: 2758309 Columns: 43
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (29): 000000000, 05, 400, 0...9, 3...13, 3...14, 3...15, 3...16, 3...17, 3...18, 3...19, 3...20, 3...21, 3...22, 3...23, 3...24, C...25, 

In [27]:
mbsf_colnames_2013_2015=c("DESY_SORT_KEY","STATE_CODE","COUNTY_CODE","SEX_CODE","RACE_CODE","AGE","ORIG_REASON_FOR_ENTITLEMENT","CURR_REASON_FOR_ENTITLEMENT","ESRD_INDICATOR","MEDICARE_STATUS_CD","PART_A_TERMINATION_CODE","PART_B_TERMINATION_CODE","ENTITLEMENT_BUY_IN_IND01","ENTITLEMENT_BUY_IN_IND02","ENTITLEMENT_BUY_IN_IND03","ENTITLEMENT_BUY_IN_IND04","ENTITLEMENT_BUY_IN_IND05","ENTITLEMENT_BUY_IN_IND06","ENTITLEMENT_BUY_IN_IND07","ENTITLEMENT_BUY_IN_IND08","ENTITLEMENT_BUY_IN_IND09","ENTITLEMENT_BUY_IN_IND10","ENTITLEMENT_BUY_IN_IND11","ENTITLEMENT_BUY_IN_IND12","HMO_INDICATOR01","HMO_INDICATOR02","HMO_INDICATOR03","HMO_INDICATOR04","HMO_INDICATOR05","HMO_INDICATOR06","HMO_INDICATOR07","HMO_INDICATOR08","HMO_INDICATOR09","HMO_INDICATOR10","HMO_INDICATOR11","HMO_INDICATOR12","HI_COVERAGE","SMI_COVERAGE","HMO_COVERAGE","STATE_BUY_IN_COVERAGE","VALID_DATE_OF_DEATH_SWITCH","DATE_OF_DEATH","REFERENCE_YEAR")

mbsf_colnames_2016_2020=c("DESY_SORT_KEY","REFERENCE_YEAR","SAMPLE_GROUP","STATE_CODE","COUNTY_CODE","STATE_CNTY_FIPS_CD_01","STATE_CNTY_FIPS_CD_02","STATE_CNTY_FIPS_CD_03","STATE_CNTY_FIPS_CD_04","STATE_CNTY_FIPS_CD_05","STATE_CNTY_FIPS_CD_06","STATE_CNTY_FIPS_CD_07","STATE_CNTY_FIPS_CD_08","STATE_CNTY_FIPS_CD_09","STATE_CNTY_FIPS_CD_10","STATE_CNTY_FIPS_CD_11","STATE_CNTY_FIPS_CD_12","SEX_CODE","RACE_CODE","AGE","ORIG_REASON_FOR_ENTITLEMENT","CURR_REASON_FOR_ENTITLEMENT","ESRD_INDICATOR","MDCR_STATUS_CODE_01","MDCR_STATUS_CODE_02","MDCR_STATUS_CODE_03","MDCR_STATUS_CODE_04","MDCR_STATUS_CODE_05","MDCR_STATUS_CODE_06","MDCR_STATUS_CODE_07","MDCR_STATUS_CODE_08","MDCR_STATUS_CODE_09","MDCR_STATUS_CODE_10","MDCR_STATUS_CODE_11","MDCR_STATUS_CODE_12","PART_A_TERMINATION_CODE","PART_B_TERMINATION_CODE","ENTITLEMENT_BUY_IN_IND01","ENTITLEMENT_BUY_IN_IND02","ENTITLEMENT_BUY_IN_IND03","ENTITLEMENT_BUY_IN_IND04","ENTITLEMENT_BUY_IN_IND05","ENTITLEMENT_BUY_IN_IND06","ENTITLEMENT_BUY_IN_IND07","ENTITLEMENT_BUY_IN_IND08","ENTITLEMENT_BUY_IN_IND09","ENTITLEMENT_BUY_IN_IND10","ENTITLEMENT_BUY_IN_IND11","ENTITLEMENT_BUY_IN_IND12","HMO_INDICATOR01","HMO_INDICATOR02","HMO_INDICATOR03","HMO_INDICATOR04","HMO_INDICATOR05","HMO_INDICATOR06","HMO_INDICATOR07","HMO_INDICATOR08","HMO_INDICATOR09","HMO_INDICATOR10","HMO_INDICATOR11","HMO_INDICATOR12","HI_COVERAGE","SMI_COVERAGE","HMO_COVERAGE","STATE_BUY_IN_COVERAGE","VALID_DATE_OF_DEATH_SWITCH","DATE_OF_DEATH","DUAL_STUS_CD_01","DUAL_STUS_CD_02","DUAL_STUS_CD_03","DUAL_STUS_CD_04","DUAL_STUS_CD_05","DUAL_STUS_CD_06","DUAL_STUS_CD_07","DUAL_STUS_CD_08","DUAL_STUS_CD_09","DUAL_STUS_CD_10","DUAL_STUS_CD_11","DUAL_STUS_CD_12")


colnames(mbsf_2013)=mbsf_colnames_2013_2015
colnames(mbsf_2014)=mbsf_colnames_2013_2015
colnames(mbsf_2015)=mbsf_colnames_2013_2015
colnames(mbsf_2016)=mbsf_colnames_2016_2020
colnames(mbsf_2017)=mbsf_colnames_2016_2020
colnames(mbsf_2018)=mbsf_colnames_2016_2020
colnames(mbsf_2019)=mbsf_colnames_2016_2020
colnames(mbsf_2020)=mbsf_colnames_2016_2020

In [28]:
mbsf_2013[,year:=2013]
mbsf_2014[,year:=2014]
mbsf_2015[,year:=2015]
mbsf_2016[,year:=2016]
mbsf_2017[,year:=2017]
mbsf_2018[,year:=2018]
mbsf_2019[,year:=2019]
mbsf_2020[,year:=2020]

In [32]:
mbsf_needed_cols = c(
    "DESY_SORT_KEY"
    ,"year"
    ,"REFERENCE_YEAR"
    ,"STATE_CODE"
    ,"COUNTY_CODE"
    ,"SEX_CODE"
    ,"RACE_CODE"
    ,"AGE"
    ,"ORIG_REASON_FOR_ENTITLEMENT"
    ,"CURR_REASON_FOR_ENTITLEMENT"
    ,"ENTITLEMENT_BUY_IN_IND01"
    ,"ENTITLEMENT_BUY_IN_IND02"
    ,"ENTITLEMENT_BUY_IN_IND03"
    ,"ENTITLEMENT_BUY_IN_IND04"
    ,"ENTITLEMENT_BUY_IN_IND05"
    ,"ENTITLEMENT_BUY_IN_IND06"
    ,"ENTITLEMENT_BUY_IN_IND07"
    ,"ENTITLEMENT_BUY_IN_IND08"
    ,"ENTITLEMENT_BUY_IN_IND09"
    ,"ENTITLEMENT_BUY_IN_IND10"
    ,"ENTITLEMENT_BUY_IN_IND11"
    ,"ENTITLEMENT_BUY_IN_IND12"
    ,"HMO_INDICATOR01"
    ,"HMO_INDICATOR02"
    ,"HMO_INDICATOR03"
    ,"HMO_INDICATOR04"
    ,"HMO_INDICATOR05"
    ,"HMO_INDICATOR06"
    ,"HMO_INDICATOR07"
    ,"HMO_INDICATOR08"
    ,"HMO_INDICATOR09"
    ,"HMO_INDICATOR10"
    ,"HMO_INDICATOR11"
    ,"HMO_INDICATOR12"
    ,"VALID_DATE_OF_DEATH_SWITCH"
    ,"DATE_OF_DEATH")

mbsf_2013=mbsf_2013[, ..mbsf_needed_cols]
mbsf_2014=mbsf_2014[, ..mbsf_needed_cols]
mbsf_2015=mbsf_2015[, ..mbsf_needed_cols]
mbsf_2016=mbsf_2016[, ..mbsf_needed_cols]
mbsf_2017=mbsf_2017[, ..mbsf_needed_cols]
mbsf_2018=mbsf_2018[, ..mbsf_needed_cols]
mbsf_2019=mbsf_2019[, ..mbsf_needed_cols]
mbsf_2020=mbsf_2020[, ..mbsf_needed_cols]

mbsf_data=rbind(mbsf_2013,mbsf_2014,mbsf_2015,mbsf_2016,mbsf_2017,mbsf_2018,mbsf_2019,mbsf_2020)

In [33]:
tail(mbsf_data)

DESY_SORT_KEY,year,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH
<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
499999741,2020,2020,19,510,1,1,59,1,1,3,3,3,3,3,3,3,3,3,3,3,3,C,C,C,C,C,C,C,C,C,C,C,C,NA,NA
499999841,2020,2020,45,610,1,1,80,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA
499999951,2020,2020,50,120,2,1,60,1,1,C,C,C,C,C,C,C,C,C,C,C,C,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA
499999961,2020,2020,38,250,1,2,82,0,0,3,3,3,3,3,3,3,3,3,3,3,3,C,C,C,C,C,C,C,C,C,C,C,C,NA,NA
499999971,2020,2020,50,070,2,1,80,0,0,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,NA,NA
499999981,2020,2020,50,160,1,1,68,1,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA


In [34]:
write_fst(mbsf_data, "/work/postresearch/Shared/Projects/Data_fst/mbsf_data_long.fst")

## Read data

In [23]:
mbsf_data = read_fst(
  "/work/postresearch/Shared/Projects/Data_fst/mbsf_data_long", as.data.table = T)

In [35]:
tail(mbsf_data)

DESY_SORT_KEY,year,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH
<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
499999741,2020,2020,19,510,1,1,59,1,1,3,3,3,3,3,3,3,3,3,3,3,3,C,C,C,C,C,C,C,C,C,C,C,C,NA,NA
499999841,2020,2020,45,610,1,1,80,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA
499999951,2020,2020,50,120,2,1,60,1,1,C,C,C,C,C,C,C,C,C,C,C,C,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA
499999961,2020,2020,38,250,1,2,82,0,0,3,3,3,3,3,3,3,3,3,3,3,3,C,C,C,C,C,C,C,C,C,C,C,C,NA,NA
499999971,2020,2020,50,070,2,1,80,0,0,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,NA,NA
499999981,2020,2020,50,160,1,1,68,1,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA


In [37]:
carrier_data_all_years = read_fst(
    "carrier_data_all_years.fst", as.data.table = T,columns=c("DESY_SORT_KEY","date"))
outpatient_data_all_years = read_fst(
    "outpatient_data_all_years.fst", as.data.table = T,columns=c("DESY_SORT_KEY","date"))
inpatient_data_all_years = read_fst(
    "inpatient_data_all_years.fst", as.data.table = T,columns=c("DESY_SORT_KEY","date"))

In [39]:
carrier_data_all_years[,year:=year(date)]
outpatient_data_all_years[,year:=year(date)]
inpatient_data_all_years[,year:=year(date)]

In [40]:
head(carrier_data_all_years)
head(outpatient_data_all_years)
head(inpatient_data_all_years)

DESY_SORT_KEY,date,year
<int>,<date>,<int>
100000015,2013-04-25,2013
100000015,2013-04-25,2013
100000015,2013-04-25,2013
100000015,2013-05-28,2013
100000015,2013-05-28,2013
100000015,2013-07-19,2013


DESY_SORT_KEY,date,year
<int>,<date>,<int>
100000015,2013-04-25,2013
100000015,2013-07-19,2013
100000015,2013-10-14,2013
100000019,2013-05-23,2013
100000019,2013-05-29,2013
100000019,2013-06-03,2013


DESY_SORT_KEY,date,year
<int>,<date>,<int>
100000099,2013-10-27,2013
100000099,2013-11-03,2013
100000315,2013-01-03,2013
100000559,2013-12-24,2013
100000905,2013-08-02,2013
100001177,2013-08-25,2013


In [45]:
all_claims_all_years=rbind(carrier_data_all_years[,.(DESY_SORT_KEY,year)],
outpatient_data_all_years[,.(DESY_SORT_KEY,year)],
inpatient_data_all_years[,.(DESY_SORT_KEY,year)])
all_claims_all_years=distinct(all_claims_all_years)%>%as.data.table()

In [54]:
all_claims_all_years[,has_claim:=T]

In [55]:
head(all_claims_all_years)

DESY_SORT_KEY,year,has_claim
<int>,<int>,<lgl>
100000015,2013,TRUE
100000019,2013,TRUE
100000053,2013,TRUE
100000099,2013,TRUE
100000129,2013,TRUE
100000203,2013,TRUE


In [48]:
mbsf_data=distinct(mbsf_data[,.(DESY_SORT_KEY,year,DATE_OF_DEATH)])%>%as.data.table()

In [51]:
mbsf_data[,DESY_SORT_KEY:=as.integer(DESY_SORT_KEY)]

In [52]:
head(mbsf_data)

DESY_SORT_KEY,year,DATE_OF_DEATH
<int>,<dbl>,<dbl>
0,2013,NA
0,2013,20131017
100000005,2013,NA
100000015,2013,NA
100000019,2013,NA
100000053,2013,NA


# MBSF with any claim
If a patient has no claims in a year (e.g., they were especially healthy), they don't show up in any claims files. Do they still show up in the denominator/MBSF file for that year? You could just check one year, e.g., 2019.

In [56]:
mbsf_and_claims=left_join(mbsf_data,all_claims_all_years,by=c("DESY_SORT_KEY","year"))%>%as.data.table()
tail(mbsf_and_claims)

DESY_SORT_KEY,year,DATE_OF_DEATH,has_claim
<int>,<int>,<dbl>,<lgl>
499999741,2020,NA,NA
499999841,2020,NA,TRUE
499999951,2020,NA,TRUE
499999961,2020,NA,NA
499999971,2020,NA,NA
499999981,2020,NA,TRUE


In [57]:
mbsf_and_claims[is.na(has_claim),has_claim:=F]
tail(mbsf_and_claims)

DESY_SORT_KEY,year,DATE_OF_DEATH,has_claim
<int>,<int>,<dbl>,<lgl>
499999741,2020,NA,FALSE
499999841,2020,NA,TRUE
499999951,2020,NA,TRUE
499999961,2020,NA,FALSE
499999971,2020,NA,FALSE
499999981,2020,NA,TRUE


### No death

In [62]:
mbsf_and_claims[is.na(DATE_OF_DEATH), sum(has_claim==T)/.N, by=year]

year,V1
<int>,<dbl>
2013,0.6440753
2014,0.6303226
2015,0.6182354
2016,0.6116243
2017,0.5963612
2018,0.5847706
2019,0.5637196
2020,0.5322756


### With death

In [63]:
mbsf_and_claims[, sum(has_claim==T)/.N, by=year]

year,V1
<int>,<dbl>
2013,0.6506294
2014,0.6370855
2015,0.6249849
2016,0.6171026
2017,0.6020538
2018,0.5902808
2019,0.5694355
2020,0.5389849


# Persistence in MBSF
How persistent is the sample from year to year? I.E., using the denominator/MBSF files (not claims, since they might disappear from claims files if very healthy), of the X number of patients who appear in 2013, how many of them appear in 2014?

In [68]:
mbsf_data[,has_mbsf:=T]

In [75]:
mbsf_data_split=split(mbsf_data,by="year")

In [96]:
find_persistence=function(n){
    
    temp=left_join(mbsf_data_split[[n]],mbsf_data_split[[n+1]],by=c("DESY_SORT_KEY"),suffix = c("_n","_m"))%>%as.data.table()
    temp[is.na(DATE_OF_DEATH_n), sum(has_mbsf_m==T,na.rm=T)/.N]
}

for (n in 1:7){
    print(find_persistence(n))
}


[1] 0.995149
[1] 0.9944701
[1] 0.9899138
[1] 0.9933424
[1] 0.9967919
[1] 0.9934043
[1] 0.9939957
